In [ ]:
import h5py
from tqdm import tqdm, tqdm_notebook
import pandas as pd
import numpy as np
import json
import pims
import os
from subprocess import Popen, PIPE
import matplotlib.pyplot as plt
from matplotlib import patches

%matplotlib inline

In [2]:
# methods from https://github.com/bochinski/iou-tracker
'''
High-Speed Tracking-by-Detection Without Using Image Information
Erik Bochinski, Volker Eiselein and Thomas Sikora
Communication System Group, Technische Universitat Berlin ¨
Einsteinufer 17, 10587 Berlin
'''
pass

# Set Parameters 

In [3]:
# track iou parameters
sigma_l = 0  # Keep all detections
sigma_h = 0  # Keep all tracks
sigma_iou = 0.05 # 0.20  # Create track if overlap is better than 20%
t_min = 0  # Keep tracks of all length

# track matching parameters
frame_range = 5
iou_thresh = sigma_iou

# Define Methods

In [4]:
def iou(bbox1, bbox2):
    """
    Calculates the intersection-over-union of two bounding boxes.
    Args:
        bbox1 (numpy.array, list of floats): bounding box in format x1,y1,x2,y2.
        bbox2 (numpy.array, list of floats): bounding box in format x1,y1,x2,y2.
    Returns:
        int: intersection-over-onion of bbox1, bbox2
    """

    bbox1 = [float(x) for x in bbox1]
    bbox2 = [float(x) for x in bbox2]

    (x0_1, y0_1, x1_1, y1_1) = bbox1
    (x0_2, y0_2, x1_2, y1_2) = bbox2

    # get the overlap rectangle
    overlap_x0 = max(x0_1, x0_2)
    overlap_y0 = max(y0_1, y0_2)
    overlap_x1 = min(x1_1, x1_2)
    overlap_y1 = min(y1_1, y1_2)

    # check if there is an overlap
    if overlap_x1 - overlap_x0 <= 0 or overlap_y1 - overlap_y0 <= 0:
        return 0

    # if yes, calculate the ratio of the overlap to each ROI size and the unified size
    size_1 = (x1_1 - x0_1) * (y1_1 - y0_1)
    size_2 = (x1_2 - x0_2) * (y1_2 - y0_2)
    size_intersection = (overlap_x1 - overlap_x0) * (overlap_y1 - overlap_y0)
    size_union = size_1 + size_2 - size_intersection

    return size_intersection / size_union

In [5]:
def track_iou(detections, sigma_l, sigma_h, sigma_iou, t_min):
    """
    Simple IOU based tracker.
    See "High-Speed Tracking-by-Detection Without Using Image Information by E. Bochinski, V. Eiselein, T. Sikora" for
    more information.
    Args:
         detections (list): list of detections per frame, usually generated by util.load_mot
         sigma_l (float): low detection threshold. Any single detection with a score below sigma_l will be thrown out.
         sigma_h (float): high detection threshold. If the track does not have one score above sigma_h, throw it out
         sigma_iou (float): IOU threshold. If IOU is less than sigma_iou, the rois don't match.
         t_min (float): minimum track length in frames.
    Returns:
        list: list of tracks.
    """

    tracks_active = []
    tracks_finished = []

    for frame_num, detections_frame in enumerate(tqdm(detections), start=1):
        # apply low threshold to detections
        dets = [det for det in detections_frame if det['score'] >= sigma_l]

        updated_tracks = []
        for track in tracks_active:
            if len(dets) > 0:
                # get det with highest iou
                best_match = max(dets, key=lambda x: iou(track['bboxes'][-1], x['bbox']))
                if iou(track['bboxes'][-1], best_match['bbox']) >= sigma_iou:
                    track['bboxes'].append(best_match['bbox'])
                    track['max_score'] = max(track['max_score'], best_match['score'])

                    updated_tracks.append(track)

                    # remove from best matching detection from detections
                    del dets[dets.index(best_match)]

            # if track was not updated
            if len(updated_tracks) == 0 or track is not updated_tracks[-1]:
                # finish track when the conditions are met
                if track['max_score'] >= sigma_h and len(track['bboxes']) >= t_min:
                    tracks_finished.append(track)

        # create new tracks
        new_tracks = [{'bboxes': [det['bbox']], 'max_score': det['score'], 'start_frame': frame_num} for det in dets]
        tracks_active = updated_tracks + new_tracks

    # finish all remaining active tracks
    tracks_finished += [track for track in tracks_active
                        if track['max_score'] >= sigma_h and len(track['bboxes']) >= t_min]

    return tracks_finished

In [7]:
# FIND MATCHING TARJECTORIES
# ------------------------------------------------
#   NEW   ----   NEW   ----   NEW   ----   NEW   ----   NEW   ----   NEW   ----   NEW
# ------------------------------------------------
def get_traj_matches(traj_df, frame_range, iou_thresh):
    '''
    Match tracks if close in time with significant overlap
    traj_df: Pandas dataframe containing trajectories to match
    frame_range: Number of frames to "remember" an object position in order to link more tracks through time if detections are spotty
    iou_thresh:  IOU threshold to determine if an object is the same between frames (same as sigma_iou)
    '''

    # Group by id and get the min/max frame for each group to create min/max dataframe
    df_traj_min_frame = traj_df[traj_df['frame']==traj_df.groupby('id')['frame'].transform(lambda x: x.min())]
    df_traj_max_frame = traj_df[traj_df['frame']==traj_df.groupby('id')['frame'].transform(lambda x: x.max())]


    df_traj_matches = pd.DataFrame()
    
    # Start with the MAX track frame to avoid duplicates.
    # The MAX frame of one trajectory should match the best MIN frame of the next.
    for i, max_row in df_traj_max_frame.iterrows():
    #     print(min_row)
        frame = max_row['frame']
        # Get all max frames that are less, but within a given frame_range
        df_min = df_traj_min_frame[(df_traj_min_frame['frame'] > frame) & (df_traj_min_frame['frame'] <= frame+frame_range)]
    #     print(df_max)
        match_score = 0
        match_id = False
        for j, min_row in df_min.iterrows():
            # Match tracks if IOU is above the threshold
            # Only use the best match
            iou_score = iou(min_row['bbox'], max_row['bbox'])
            if (iou_score > iou_thresh) & (iou_score > match_score):
                match_score = iou_score
                match_id = min_row['id']
        if match_id:
            min_id=max_row['id']
            # Trace back to the min track id
            while True:
                if len(df_traj_matches) == 0:
                    break
                nid = df_traj_matches[df_traj_matches['id2']==min_id]['id1']
                if nid.count() > 0:
                    min_id = nid.values[0]
                    #print(min_id)
                else:
                    break
            df_traj_matches = df_traj_matches.append({'min_id':min_id, 'id1':max_row['id'], 'id2':match_id, 'frame':frame, 'score':match_score}, ignore_index=True)
    #         print('match_score',match_score)
    #         print('match_frame',match_frame)
    #         print('match_id',match_id)
    #     else:
    #         print('None')

    #     print('-'*20)
    return df_traj_matches

In [8]:
def join_matched_trajectories(df_orig_traj, df_traj_matches):
    '''
    Join all matched tracks by changing their ids in the dataframe.
    Store original ids in a new 'orig_id' column and add new ids in 
    the 'id' column.
    df_orig_traj: Pandas dataframe with columns ['bbox', 'frame', 'id']
    df_traj_matches: Pandas dataframe matching old ids to new ids. 
                Columns are ['frame', 'id1', 'id2', 'min_id', 'score']
    return: Original dataframe with new id column representing matched 
            trajectories.
    '''

    print(' '*6,'Tracks before matching:', len(df_orig_traj['id'].unique()))
    
    # Copy the original dataframe so we don't change it
    df_matched = df_orig_traj.copy()
    
    # Make a new column to store the original ids
    df_matched['orig_id'] = df_matched['id']
    
    # Create a map: All ids's matching id2 should be replaced by min_id
    mapping = {x:y for x,y in zip(df_traj_matches['id2'], df_traj_matches['min_id'])}
    
    # Place all mached ids into the 'id' column
    df_matched['id'] = df_matched['id'].map(mapping)
    df_matched['id'].fillna(df_matched['orig_id'], inplace=True)  # Fill NAN values (not mapped) with the original id
    
    print(' '*6,'Tracks after matching: ', len(df_matched['id'].unique()))
    
    return df_matched

In [9]:
def organize_trajectories(trajectories):
    '''
    Take a list of trajectories output by track_iou() and organize them into a 
    pandas dataframe sorted by frame and trajectory id.
    trajectories: List of trajectories output by track_iou() where each element
                of the list consists of a dictionary of {bboxes, score, start_frame}
    return: Pandas dataframe sorted by frame and trajectory id.
    '''
    
    print(' '*6,'Organize trajectories')
    
    # Create a list to store rows of [frame, id, bbox]
    traj_list = []    
    
    # Reorganize array of trajectory dicts into new dataframe
    for traj_id, traj in enumerate(tqdm(trajectories)):
        l= len(traj['bboxes'])
        f = traj['start_frame']
        for bbox in traj['bboxes']:
            # Append to list (fast) rather than dataframe (VERY SLOW)
            traj_list.append([f, traj_id, bbox])
            f +=1
    
    # Convert 2d list into dataframe
    df_traj = pd.DataFrame(traj_list, columns=['frame', 'id', 'bbox'])
    
    # Sort dataframe by 'frame' and 'id' and reset the index
    df_traj_sorted = df_traj.sort_values(['frame','id'])
    df_traj_sorted = df_traj_sorted.reset_index(drop=True)
    
    return df_traj_sorted

In [10]:
def create_trajectories(json_path, save_orig_traj=True, save_final_traj=True):
    '''
    Main function calling other subfunctions used to take raw bounding box
    detections in JSON format and create a pandas dataframe of object 
    trajectories.
    Optionally save trajectories as .csv files in the same location as the
    original file, appending "_traj.csv" for original IOU trajectories and
    "_traj_match.csv" for final matched trajectories.
    json_path: Path to JSON file containing a list of detections per frame.
    save_orig_traj: If True, save the original trajectories as a CSV file.
    save_final_traj: If True, save the final matched trajectories to CSV.
    return: Pandas dataframe of 
    '''
    
    # Load bbox detections from json
    with open(json_path) as f:
        detections = json.load(f)

    # Create IOU trajectories dictionary
    print(' '*6,'Create trajectories')
    trajectories = track_iou(detections, sigma_l=sigma_l, sigma_h=sigma_h, sigma_iou=sigma_iou, t_min=t_min)

    # Organize trajectories by id and frame
    df_traj_sorted = organize_trajectories(trajectories)
    
    if save_orig_traj:
        # Save original IOU trajectories
        save_name = json_path.replace('.json','_traj.csv')
        df_traj_sorted.to_csv(save_name)

    ##################
    # Append tracks if close in time with significant overlap
    ##################

    # Find matching tracks
    df_traj_matches = get_traj_matches(df_traj_sorted, frame_range, iou_thresh)

    # Append matching tracks by changing the track id
    df_traj_sorted_matched = join_matched_trajectories(df_traj_sorted, df_traj_matches)
    
    if save_final_traj:
        save_name = json_path.replace('.json','_traj_match.csv')
        df_traj_sorted_matched.to_csv(save_name)
    
    return df_traj_sorted_matched, df_traj_matches

# BATCH PROCESS

In [ ]:
for root, dirs, files in os.walk('Drer_MC3R_Behavior/'):
    if '.ipynb_checkpoints' in root:
        continue
    if root < 'Drer_MC3R_Behavior/20181115':
        continue
    print(root)
    files.sort()
    for file in files:
        if not file.endswith('_detections.json'):
            continue
        # Get full path
        json_path = os.path.join(root, file)
        print(' '*4,json_path)
        
        # Create trajectories from json detections and save as CSV
        df_trajectories, df_matches = create_trajectories(json_path, save_orig_traj=True, save_final_traj=True)
print('-'*20, 'DONE', '-'*20)

# SINGLE FILE

In [ ]:
file = 'Drer_MC3R_Behavior/20181115/Control_20181115-1030_detections.json'
df_trajectories, df_matches = create_trajectories(file, False, False)

In [ ]:
file = 'Drer_MC3R_Behavior/20181115/Control_20181115-1030_detections.json'
df_trajectories, df_matches = create_trajectories(file, False, False)

# Check all trajectories for duplicates

In [ ]:
for root, dirs, files in os.walk('Drer_MC3R_Behavior/'):
    if '.ipynb_checkpoints' in root:
        continue
    if root < 'Drer_MC3R_Behavior/20181115':
        continue  # ONLY DAYS > 11/15
    print(root)
    files.sort()
    for file in files:
        if not file.endswith('_traj_match.csv'):
            continue
        # Get full path
        path = os.path.join(root, file)
        print(path)
        
        # Load trajectories
        with open(path) as f:
            df_trajectories = pd.read_csv(f)
        dups = len(df_trajectories[df_trajectories.duplicated(subset=['frame','id'])])
        print(' '*4,'Duplicates:',dups)
print('-'*20, 'DONE', '-'*20)

In [41]:
df_trajectories[df_trajectories.duplicated(subset=['frame','id'])]

,frame,id,bbox,orig_id


In [35]:
len(df_trajectories['id'].unique())

1356

In [33]:
df_matches[:50]

,frame,id1,id2,min_id,score
0,6.0,0.0,2.0,0.0,0.601266
1,57.0,2.0,6.0,0.0,0.313953
2,60.0,3.0,6.0,3.0,0.757009
3,62.0,4.0,5.0,4.0,0.899471
4,114.0,6.0,8.0,0.0,0.575000
5,120.0,7.0,9.0,7.0,0.367347
6,120.0,8.0,16.0,0.0,0.373980
7,125.0,9.0,11.0,7.0,0.604766
8,219.0,12.0,15.0,12.0,0.388430
9,220.0,13.0,14.0,13.0,0.421520


In [30]:
df_matches[:50]

,frame,id1,id2,min_id,score
0,60.0,2.0,3.0,2.0,0.291971
1,62.0,3.0,6.0,2.0,0.757009
2,64.0,4.0,5.0,4.0,0.899471
3,118.0,6.0,8.0,2.0,0.575000
4,122.0,8.0,16.0,2.0,0.373980
5,123.0,7.0,9.0,7.0,0.367347
6,127.0,9.0,11.0,7.0,0.604766
7,222.0,12.0,15.0,12.0,0.388430
8,223.0,13.0,14.0,13.0,0.421520
9,242.0,16.0,17.0,2.0,0.752549


In [29]:
df_matches[(df_matches.id1==177) | (df_matches.id2==177)]

,frame,id1,id2,min_id,score
108,3287.0,175.0,177.0,171.0,0.177879
109,3290.0,177.0,180.0,171.0,0.093656
110,3293.0,177.0,179.0,171.0,0.371060


In [24]:
df_matches[(df_matches.frame>=3290) & (df_matches.frame<=3295)]

,frame,id1,id2,min_id,score
109,3290.0,177.0,180.0,171.0,0.093656
110,3293.0,177.0,179.0,171.0,0.371060


In [20]:
df_trajectories[df_trajectories.orig_id==180][:10]

,frame,id,bbox,orig_id
15943,3290,171.0,"[226, 136, 250, 207]",180
15947,3291,171.0,"[223, 150, 243, 220]",180
15951,3292,171.0,"[219, 173, 237, 236]",180
15956,3293,171.0,"[218, 183, 238, 254]",180
15961,3294,171.0,"[215, 192, 236, 266]",180
15966,3295,171.0,"[215, 202, 237, 278]",180
15971,3296,171.0,"[215, 213, 239, 287]",180
15976,3297,171.0,"[213, 228, 242, 292]",180
15981,3298,171.0,"[209, 244, 249, 296]",180
15986,3299,171.0,"[204, 257, 253, 300]",180


In [18]:
df_trajectories[df_trajectories.orig_id==179]

,frame,id,bbox,orig_id
15955,3293,171.0,"[249, 132, 288, 187]",179
15960,3294,171.0,"[237, 135, 290, 182]",179
15965,3295,171.0,"[230, 141, 275, 165]",179
15970,3296,171.0,"[228, 142, 268, 165]",179
15975,3297,171.0,"[225, 144, 272, 166]",179
15980,3298,171.0,"[220, 145, 269, 167]",179
15985,3299,171.0,"[215, 146, 268, 169]",179
15990,3300,171.0,"[216, 148, 263, 171]",179
15995,3301,171.0,"[207, 149, 258, 170]",179
16000,3302,171.0,"[208, 150, 258, 172]",179


In [16]:
df_trajectories[(df_trajectories['frame']>=3290) & (df_trajectories['frame']<=3295)]

,frame,id,bbox,orig_id
15942,3290,178.0,"[264, 175, 281, 215]",178
15943,3290,171.0,"[226, 136, 250, 207]",180
15944,3290,182.0,"[215, 194, 239, 266]",182
15945,3290,190.0,"[0, 0, 30, 44]",190
15946,3290,167.0,"[267, 140, 295, 214]",207
15947,3291,171.0,"[223, 150, 243, 220]",180
15948,3291,182.0,"[215, 218, 241, 284]",182
15949,3291,190.0,"[0, 0, 30, 44]",190
15950,3291,167.0,"[262, 143, 288, 217]",207
15951,3292,171.0,"[219, 173, 237, 236]",180


In [14]:
df_trajectories[df_trajectories.duplicated(subset=['frame','id'], keep=False)]

,frame,id,bbox,orig_id
15955,3293,171.0,"[249, 132, 288, 187]",179
15956,3293,171.0,"[218, 183, 238, 254]",180
15960,3294,171.0,"[237, 135, 290, 182]",179
15961,3294,171.0,"[215, 192, 236, 266]",180
15965,3295,171.0,"[230, 141, 275, 165]",179
15966,3295,171.0,"[215, 202, 237, 278]",180
15970,3296,171.0,"[228, 142, 268, 165]",179
15971,3296,171.0,"[215, 213, 239, 287]",180
15975,3297,171.0,"[225, 144, 272, 166]",179
15976,3297,171.0,"[213, 228, 242, 292]",180


In [15]:
df_trajectories.describe()

,frame,id,orig_id
count,304123.000000,304123.000000,304123.000000
mean,31571.448351,1451.543984,1814.846020
std,18264.799346,1155.778844,1076.202908
min,1.000000,0.000000,0.000000
25%,15709.500000,190.000000,854.000000
50%,31492.000000,1226.000000,1722.000000
75%,47435.500000,2557.000000,2754.000000
max,63172.000000,3712.000000,3714.000000


In [ ]:
# TESTING

file = 'Drer_MC3R_Behavior/20181115/Control_20181115-1030_detections.json'

# Load bbox detections from json
with open(file) as f:
    detections = json.load(f)

# Create IOU trajectories dictionary
print(' '*6,'Create trajectories')
trajectories = track_iou(detections, sigma_l=sigma_l, sigma_h=sigma_h, sigma_iou=sigma_iou, t_min=t_min)

# Organize trajectories by id and frame
df_traj = organize_trajectories(trajectories)

# # Save original IOU trajectories
# save_name = os.path.join(root,file.replace('.json','_traj.csv'))
# df_traj_sorted.to_csv(save_name)

##################
# Append tracks if close in time with significant overlap
##################

# Find matching tracks
df_traj_matches = get_traj_matches(df_traj_sorted, frame_range, iou_thresh)

# Append matching tracks by changing the track id
df_traj_sorted_matched = join_matched_trajectories(df_traj_sorted, df_traj_matches)

In [ ]:
##################
# Append tracks if close in time with significant overlap
##################

# Find matching tracks
df_traj_matches = get_traj_matches(df_traj_sorted, frame_range, iou_thresh)

# Append matching tracks by changing the track id
df_traj_sorted_matched = df_traj_sorted.copy()
print(' '*6,'Tracks before matching:', len(df_traj_sorted_matched['id'].unique()))
for i, row in tqdm(df_traj_matches.iterrows(), total=df_traj_matches.shape[0]):
    r_id = row['id2']  # ID to replace
    min_id = row['min_id']  # New id value
    # replace all matching r_id with min_id
    df_traj_sorted_matched['id'][df_traj_sorted_matched['id']==r_id] = min_id
print('Tracks after matching:', len(df_traj_sorted_matched['id'].unique()))

In [114]:
df_tst = join_matched_trajectories(df_traj_sorted, df_traj_matches)

       Tracks before matching: 1268
       Tracks after matching:  1268


In [101]:
df_tsm = df_traj_sorted.copy()
df_tsm['orig_id'] = df_tsm['id']  # Make a new column to store the original ids
# Create a map: All ids's matching id2 should be replaced by min_id
mapping = {x:y for x,y in zip(df_traj_matches['id2'], df_traj_matches['min_id'])}
df_tsm['id'] = df_tsm['id'].map(mapping)
df_tsm['id'].fillna(df_tsm['orig_id'], inplace=True)  # Fill NAN values (not mapped) with the original id

In [107]:
df_traj_sorted.columns

Index(['bbox', 'frame', 'id'], dtype='object')

In [102]:
df_tsm.describe()

,frame,id,orig_id
count,304123.000000,304123.000000,304123.000000
mean,31571.448351,1451.543984,1814.846020
std,18264.799346,1155.778844,1076.202908
min,1.000000,0.000000,0.000000
25%,15709.500000,190.000000,854.000000
50%,31492.000000,1226.000000,1722.000000
75%,47435.500000,2557.000000,2754.000000
max,63172.000000,3712.000000,3714.000000


In [103]:
df_tsm.head()

,bbox,frame,id,orig_id
0,"[369, 132, 389, 181]",1.0,0.0,0.0
1,"[270, 161, 291, 244]",1.0,1.0,1.0
2,"[346, 288, 368, 373]",1.0,7.0,7.0
3,"[324, 122, 356, 206]",1.0,20.0,20.0
4,"[0, 0, 31, 44]",1.0,190.0,190.0


In [45]:
# Find matching tracks
df_traj_matches_frames = get_traj_matches(df_traj_sorted, frame_range, iou_thresh)

In [47]:
df_t_match = df_traj_sorted.copy()

In [49]:
df_t_match['match_id'] = df_t_match['id']


In [50]:
df_t_match.head()

,bbox,frame,id,match_id
0,"[369, 132, 389, 181]",1.0,0.0,0.0
1,"[270, 161, 291, 244]",1.0,1.0,1.0
2,"[346, 288, 368, 373]",1.0,7.0,7.0
3,"[324, 122, 356, 206]",1.0,20.0,20.0
4,"[0, 0, 31, 44]",1.0,190.0,190.0


In [108]:
df_traj_matches_frames.columns

Index(['frame', 'id1', 'id2', 'min_id', 'score'], dtype='object')

In [46]:
df_traj_matches_frames[df_traj_matches_frames['min_id']==171]

,frame,id1,id2,min_id,score
105,3213.0,171.0,172.0,171.0,0.294092
106,3249.0,172.0,175.0,171.0,0.407407
108,3287.0,175.0,177.0,171.0,0.177879
109,3290.0,177.0,180.0,171.0,0.093656
110,3293.0,177.0,179.0,171.0,0.371060
111,3304.0,179.0,208.0,171.0,0.333333
112,3387.0,180.0,185.0,171.0,0.681818
116,3479.0,185.0,200.0,171.0,0.526316
129,4007.0,208.0,211.0,171.0,0.202917


In [43]:
df_traj_matches[df_traj_matches['min_id']==171]

,id1,id2,min_id,score
102,171.0,172.0,171.0,0.294092
105,172.0,175.0,171.0,0.407407
107,175.0,177.0,171.0,0.177879
108,177.0,179.0,171.0,0.371060
109,177.0,180.0,171.0,0.093656
112,180.0,185.0,171.0,0.681818
120,185.0,200.0,171.0,0.526316
124,179.0,208.0,171.0,0.333333
127,208.0,211.0,171.0,0.202917


In [35]:
traj_df[:10]

,bbox,frame,id
0,"[369, 132, 389, 181]",1.0,0.0
1,"[368, 130, 389, 178]",2.0,0.0
2,"[367, 129, 387, 177]",3.0,0.0
3,"[366, 130, 388, 178]",4.0,0.0
4,"[360, 134, 388, 175]",5.0,0.0
5,"[363, 135, 383, 176]",6.0,0.0
6,"[270, 161, 291, 244]",1.0,1.0
7,"[272, 151, 291, 235]",2.0,1.0
8,"[274, 144, 292, 231]",3.0,1.0
9,"[274, 137, 293, 225]",4.0,1.0


In [22]:
trajectories[:10]

[{'bboxes': [[369, 132, 389, 181],
   [368, 130, 389, 178],
   [367, 129, 387, 177],
   [366, 130, 388, 178],
   [360, 134, 388, 175],
   [363, 135, 383, 176]],
  'max_score': 0.9969947338104248,
  'start_frame': 1},
 {'bboxes': [[270, 161, 291, 244],
   [272, 151, 291, 235],
   [274, 144, 292, 231],
   [274, 137, 293, 225],
   [275, 132, 293, 221],
   [274, 128, 293, 215],
   [274, 124, 293, 211],
   [274, 121, 291, 206],
   [274, 112, 290, 204],
   [274, 107, 291, 198],
   [274, 101, 290, 192],
   [273, 99, 290, 191],
   [273, 94, 290, 188],
   [273, 95, 291, 188],
   [271, 95, 289, 175],
   [271, 97, 289, 168],
   [269, 102, 288, 158],
   [269, 103, 287, 155]],
  'max_score': 0.9999324083328247,
  'start_frame': 1},
 {'bboxes': [[360, 130, 388, 173],
   [365, 128, 389, 176],
   [368, 126, 390, 178],
   [368, 126, 391, 179],
   [369, 126, 392, 183],
   [368, 126, 393, 187],
   [369, 125, 395, 190],
   [369, 124, 396, 195],
   [369, 124, 396, 197],
   [371, 124, 397, 199],
   [372, 12

In [33]:
df_traj_sorted[(df_traj_sorted['frame']>=3293) & (df_traj_sorted['frame']<=3300)]

,bbox,frame,id
15955,"[249, 132, 288, 187]",3293.0,179.0
15956,"[218, 183, 238, 254]",3293.0,180.0
15957,"[219, 263, 243, 323]",3293.0,182.0
15958,"[0, 0, 30, 45]",3293.0,190.0
15959,"[263, 173, 294, 243]",3293.0,207.0
15960,"[237, 135, 290, 182]",3294.0,179.0
15961,"[215, 192, 236, 266]",3294.0,180.0
15962,"[221, 272, 244, 349]",3294.0,182.0
15963,"[0, 0, 30, 45]",3294.0,190.0
15964,"[265, 185, 297, 248]",3294.0,207.0


In [ ]:
df_traj_sorted_matched[df_traj_sorted_matched.duplicated(subset=['frame','id'],keep=False)]

In [ ]:
df_traj_sorted_matched[(df_traj_sorted_matched['frame']>=3293) & (df_traj_sorted['frame']<=3300)]

In [ ]:
df_traj_sorted_matched[(df_traj_sorted_matched['frame']>=4030) & (df_traj_sorted['frame']<=4040)]

In [ ]:
df_traj_sorted[(df_traj_sorted['frame']>=4030) & (df_traj_sorted['frame']<=4040)]

# Create Trajectories

In [ ]:
colors = create_color_cycle()

for root, dirs, files in os.walk('Drer_MC3R_Behavior/20181116/'):
    if '.ipynb_checkpoints' in root:
        continue
    print(root)
    files.sort()
    for file in files:
        if not file.endswith('.json'):
            continue
        
        print(' '*4,file)
        # Load bbox detections from json
        with open(os.path.join(root,file)) as f:
            detections = json.load(f)

        # Create IOU trajectories
        print(' '*6,'Create trajectories')
        trajectories = track_iou(detections, sigma_l=sigma_l, sigma_h=sigma_h, sigma_iou=sigma_iou, t_min=t_min)

        # Organize trajectories by id and frame
        print(' '*6,'Organize trajectories')
        traj_df = pd.DataFrame()
        for traj_id, traj in enumerate(tqdm(trajectories)):
            l= len(traj['bboxes'])
            f = traj['start_frame']
            for bbox in traj['bboxes']:
                traj_df = traj_df.append({'frame':f, 'id':traj_id, 'bbox':bbox}, ignore_index=True)
                f +=1

        df_traj_sorted = traj_df.sort_values(['frame','id'])
        df_traj_sorted = df_traj_sorted.reset_index(drop=True)

        # Save original IOU trajectories
        save_name = os.path.join(root,file.replace('.json','_traj.csv'))
        df_traj_sorted.to_csv(save_name)

        ##################
        # Append tracks if close in time with significant overlap
        ##################

        # Find matching tracks
        df_traj_matches = get_traj_matches(frame_range, iou_thresh)

        # Append matching tracks by changing the track id
        df_traj_sorted_matched = df_traj_sorted.copy()
        print(' '*6,'Tracks before matching:', len(df_traj_sorted_matched['id'].unique()))
        for i, row in tqdm(df_traj_matches.iterrows(), total=df_traj_matches.shape[0]):
            r_id = row['id2']  # ID to replace
            min_id = row['min_id']  # New id value
            # replace all matching r_id with min_id
            df_traj_sorted_matched['id'][df_traj_sorted_matched['id']==r_id] = min_id
        print('Tracks after matching:', len(df_traj_sorted_matched['id'].unique()))
        save_name = save_name.replace('.csv','_match.csv')
        df_traj_sorted_matched.to_csv(save_name)
        

# Output matched tracks to video

In [330]:
# Pipe output directly to mp4 using ffmpeg
video_name = 'video_linked_f5_iou20_result_iou20_t3_l0_h0.mp4'
fps = mp4.frame_rate
with Popen(['ffmpeg', '-y', '-f', 'image2pipe', '-vcodec', 'mjpeg', '-r', str(fps), '-i', '-', '-vcodec', 'mpeg4', '-qscale', '5', '-r', str(fps), video_name], stdin=PIPE) as p:
    create_overlay_image_trackpy(mp4[:1000], df_traj_sorted_matched, colors, detections=detections, ffmpeg_pipe=p.stdin, fps=fps, burn_frame=True)
# p.stdin.close()
# p.wait()

100%|██████████| 1000/1000 [01:10<00:00, 14.12it/s]


In [326]:
df_traj_sorted_matched[df_traj_sorted_matched['frame'].isin(np.arange(348,353))]

,bbox,frame,id
1651,"(209, 267, 286, 309)",348.0,35.0
1652,"(554, 311, 597, 347)",348.0,19.0
1653,"(316, 337, 408, 365)",348.0,13.0
1654,"(88, 204, 160, 236)",348.0,32.0
1655,"(218, 271, 289, 312)",349.0,35.0
1656,"(547, 309, 599, 351)",349.0,19.0
1657,"(311, 334, 404, 363)",349.0,13.0
1658,"(85, 205, 163, 236)",349.0,32.0
1659,"(225, 276, 296, 314)",350.0,35.0
1660,"(74, 198, 112, 257)",350.0,32.0


In [ ]:
df_traj_matches

In [325]:
df_traj_sorted[df_traj_sorted['frame'].isin(np.arange(348,353))]

,bbox,frame,id
1651,"(209, 267, 286, 309)",348.0,35.0
1652,"(554, 311, 597, 347)",348.0,37.0
1653,"(316, 337, 408, 365)",348.0,43.0
1654,"(88, 204, 160, 236)",348.0,53.0
1655,"(218, 271, 289, 312)",349.0,35.0
1656,"(547, 309, 599, 351)",349.0,37.0
1657,"(311, 334, 404, 363)",349.0,43.0
1658,"(85, 205, 163, 236)",349.0,53.0
1659,"(225, 276, 296, 314)",350.0,35.0
1660,"(74, 198, 112, 257)",350.0,36.0


In [183]:
df_traj_mm = traj_df
df_traj_mm['frame_min'] = df_traj_mm['frame']
df_traj_mm = df_traj_mm.rename(columns={'frame':'frame_max'})
print(df_traj_mm.head())
tst = df_traj_mm.groupby('id').agg({'bbox':'last', 'frame_min':'min', 'bbox':'last', 'frame_max':'max'})
print(tst.head())
tst = tst.stack()
print(tst.head())
tst[0]['frame_max']

                   bbox  frame_max   id  frame_min
0   (96, 282, 119, 332)       13.0  0.0       13.0
1   (95, 274, 118, 327)       14.0  0.0       14.0
2   (95, 263, 119, 334)       15.0  0.0       15.0
3  (132, 340, 189, 400)        1.0  1.0        1.0
4  (131, 329, 182, 397)        2.0  1.0        2.0
                     bbox  frame_min  frame_max
id                                             
0.0   (95, 263, 119, 334)       13.0       15.0
1.0  (116, 266, 169, 351)        1.0       16.0
2.0  (106, 221, 158, 298)       18.0       21.0
3.0   (81, 207, 117, 276)       17.0       25.0
4.0  (241, 301, 264, 335)        1.0       34.0
id            
0.0  bbox          (95, 263, 119, 334)
     frame_min                      13
     frame_max                      15
1.0  bbox         (116, 266, 169, 351)
     frame_min                       1
dtype: object


15.0

In [159]:
traj_df.groupby('id').min()['frame']

id
0.0        13.0
1.0         1.0
2.0        18.0
3.0        17.0
4.0         1.0
5.0         1.0
6.0        34.0
7.0        23.0
8.0        56.0
9.0         1.0
10.0       38.0
11.0       68.0
12.0       81.0
13.0       92.0
14.0       86.0
15.0        1.0
16.0      130.0
17.0       38.0
18.0       69.0
19.0      193.0
20.0       97.0
21.0      137.0
22.0       98.0
23.0      244.0
24.0      285.0
25.0      296.0
26.0      266.0
27.0      301.0
28.0      317.0
29.0      315.0
          ...  
126.0    1540.0
127.0    1601.0
128.0    1516.0
129.0    1652.0
130.0    1570.0
131.0    1671.0
132.0    1660.0
133.0    1682.0
134.0    1675.0
135.0    1479.0
136.0    1529.0
137.0    1642.0
138.0    1709.0
139.0    1717.0
140.0    1742.0
141.0    1696.0
142.0    1722.0
143.0    1779.0
144.0    1786.0
145.0    1749.0
146.0    1719.0
147.0    1846.0
148.0    1853.0
149.0    1716.0
150.0    1889.0
151.0    1756.0
152.0    1840.0
153.0    1875.0
154.0    1927.0
155.0    1936.0
Name: frame, Length: 